In [1]:
import os
import csv
import numpy as np

In [2]:
main_dir ="/Users/08volt/Desktop/StremingML/TEST/Results_union"
old_drifts = ["sea_P(y)","sea_P(y_x)","sea_P(x_y)","sine_P(y)","sine_P(y_x)","sine_P(x_y)"]
new_drifts = ["jitter","disappearing-minority","appearing-minority","appearing-clusters",
             "shapeshift","minority-share","splitting-clusters","clusters-movement","borderline"]
type_drift = ["incremental","sudden","periodic"]
unbalance_lvls = [1,2,3,4]
stats_w = ["Recall_0","Recall_1", "F1_0", "F1_1", "Gmean"]
stats_tot =["time", "memory"]
algs = ["RebalanceStream","CSMOTE","improved_UOB","improved_OOB","WEOB1","WEOB2","ESOS_ELM","UOB","OOB"]

In [3]:
def import_csv(csvfilename):
    data = []
    with open(csvfilename, "r") as scraped:
        reader = csv.reader(scraped, delimiter=',')
        row_index = 0
        for row in reader:
            if row:  # avoid blank lines
                row_index += 1
                columns = [str(row_index)]+ [row[i] for i in range(len(row))]
                data.append(columns)
        #print(f"{row_index} valid rows")
        
    return data

# MEAN ALL ROWS

# OLD

In [4]:
final_dict = {}

for old in old_drifts:
    path = main_dir + "/" + old[old.index("P"):]   #old = sea_P(y)
    if old not in final_dict:
        final_dict[old] = {}
        
    for ds in os.listdir(path):
        #check if ds in old (directory giusta)
        if old[:3] == ds[:3]:
            path1 = path + "/" + ds
            
            unbalance = 0
            for u in unbalance_lvls:
                if str(u) in ds:
                    unbalance = u
            assert unbalance != 0
                        
            if unbalance not in final_dict[old]:
                final_dict[old][unbalance] = {}
                
            dtype = "sudden"
            if "G" in ds:
                dtype = "incremental"
                
            if dtype not in final_dict[old][unbalance]:
                final_dict[old][unbalance][dtype] = {}

            for alg in os.listdir(path1):
                if "DS_Store" in alg:
                    continue
                path2 = path1 + "/" + alg
                class0 = []
                class1 = []
                
                print(f"{old} {unbalance} {dtype} {alg}")
                #prendo i valori di tutti i 10 esperimenti
                for result in os.listdir(path2):
                    if "err" not in result or "DS_Store" in result:
                        continue
                    start = False
                    data = import_csv(path2+"/"+ result)
                    daily0 = []
                    daily1 = []
                    for data_row in data:
                        if data_row[1] == "learning evaluation instances":
                            start = True
                            continue
                        if not start:
                            continue
                        
                        try:
                            daily0.append(float(data_row[11]))
                        except:
                            if data_row[11] == "?":
                                daily0.append(0)
                            else:
                                print("ZERO")
                                print(data_row)
                        try:
                            daily1.append(float(data_row[12]))
                        except:
                            if data_row[12] == "?":
                                daily1.append(0)
                            else:
                                print("ONE")
                                print(data_row)
                    if len(daily0)>0 and len(daily1)>0:
                        class0.append(np.mean(daily0))
                        class1.append(np.mean(daily1))
                            
                        
                    
                assert len(class0) > 0
                assert len(class1) > 0
                assert len(class1) == len(class0)
                if np.mean(class0) != np.mean(class0):
                    print(class0)
                if np.mean(class1) != np.mean(class1):
                    print(class1)
                if alg == "OSHAT":
                    alg = "CSMOTE"
                final_dict[old][unbalance][dtype][alg] = {}
                final_dict[old][unbalance][dtype][alg]["class0"] ={} 
                final_dict[old][unbalance][dtype][alg]["class1"] = {}
                final_dict[old][unbalance][dtype][alg]["class0"]["mean"] = np.mean(class0)
                final_dict[old][unbalance][dtype][alg]["class1"]["mean"] = np.mean(class1)
                final_dict[old][unbalance][dtype][alg]["class0"]["std"] =np.std(class0)
                final_dict[old][unbalance][dtype][alg]["class1"]["std"] = np.std(class1)

                print(np.mean(class0))
                print(np.mean(class1))
                    
                    
                
        

sea_P(y) 1 sudden improved_OOB
98.48561710677316
99.49352354854804
sea_P(y) 1 sudden WEOB1
98.33357405890271
99.6169731540414
sea_P(y) 1 sudden OOB
98.3973579253213
99.44375523661355
sea_P(y) 1 sudden ESOS_ELM
97.36586030796744
74.88531568967352
sea_P(y) 1 sudden improved_UOB
98.06879810369382
99.5261267858681
sea_P(y) 1 sudden WEOB2
97.74454272188926
99.44302542805681
sea_P(y) 1 sudden UOB
98.3973579253213
99.44375523661355
sea_P(y) 1 sudden OSHAT
98.1380466115811
99.1298111232395
sea_P(y) 1 sudden RebalanceStream
98.50947220175786
90.67423325232042
sea_P(y) 3 incremental improved_OOB
93.13412765883939
74.15291431408767
sea_P(y) 3 incremental WEOB1
92.74288259658843
74.64764045740522
sea_P(y) 3 incremental OOB
92.01289268955256
75.05212314081929
sea_P(y) 3 incremental ESOS_ELM
92.38578205575467
57.98461674609191
sea_P(y) 3 incremental improved_UOB
92.01313159518239
74.83421749655042
sea_P(y) 3 incremental WEOB2
90.79551639475541
75.13802702297082
sea_P(y) 3 incremental UOB
91.79183987

78.32134495152448
sea_P(y_x) 2 incremental WEOB1
84.71730741874072
78.71671987653417
sea_P(y_x) 2 incremental OOB
84.59263130105033
80.84078678914759
sea_P(y_x) 2 incremental ESOS_ELM
86.07663803580907
59.9741486560089
sea_P(y_x) 2 incremental improved_UOB
81.99806303128449
77.85155588342136
sea_P(y_x) 2 incremental WEOB2
82.9384482796464
72.42671645119171
sea_P(y_x) 2 incremental UOB
77.92632683869383
80.50307285016962
sea_P(y_x) 2 incremental OSHAT
83.21137752383491
72.57395899169829
sea_P(y_x) 2 incremental RebalanceStream
94.91013946780456
44.1613007498255
sea_P(y_x) 4 sudden improved_OOB
84.29902185199371
79.50928374143278
sea_P(y_x) 4 sudden WEOB1
84.11194103267259
79.45200426739518
sea_P(y_x) 4 sudden OOB
82.10976963647047
83.19071026343394
sea_P(y_x) 4 sudden ESOS_ELM
86.61376944469342
59.51475809676761
sea_P(y_x) 4 sudden improved_UOB
83.41238339712075
79.1282570510406
sea_P(y_x) 4 sudden WEOB2
86.28930277552914
65.21165047416082
sea_P(y_x) 4 sudden UOB
79.67942389074202
83.06

99.33549719646616
97.87202717369567
sine_P(y) 1 incremental OOB
99.34737062245222
97.45397896362985
sine_P(y) 1 incremental ESOS_ELM
96.93695435247844
75.65654654556832
sine_P(y) 1 incremental improved_UOB
97.96261307078935
97.35105029289855
sine_P(y) 1 incremental WEOB2
97.84442627149427
97.53709087436684
sine_P(y) 1 incremental UOB
97.45588257219052
97.58364912356429
sine_P(y) 1 incremental OSHAT
96.38528329794917
94.13376155926537
sine_P(y) 1 incremental RebalanceStream
97.78385260575865
89.8033114517817
sine_P(y) 2 sudden improved_OOB
99.3571459962557
98.55417350898571
sine_P(y) 2 sudden WEOB1
99.10867075783626
98.73067985204321
sine_P(y) 2 sudden OOB
99.18420462502249
98.87523694872047
sine_P(y) 2 sudden ESOS_ELM
97.62750769898139
74.57195892356806
sine_P(y) 2 sudden improved_UOB
98.28731948496679
98.35303317977085
sine_P(y) 2 sudden WEOB2
98.0599484435389
98.08023923967298
sine_P(y) 2 sudden UOB
97.56788076570238
98.8599828586017
sine_P(y) 2 sudden OSHAT
95.09359393926977
98.0336

99.54092992817513
96.95295553360657
sine_P(x_y) 1 sudden ESOS_ELM
97.96094965627675
71.82848688392866
sine_P(x_y) 1 sudden improved_UOB
98.22919909715674
97.82362385565462
sine_P(x_y) 1 sudden WEOB2
97.2314193066835
97.76387014102944
sine_P(x_y) 1 sudden UOB
97.38140276110762
98.08417252573324
sine_P(x_y) 1 sudden OSHAT
96.9225173621081
91.26211321782476
sine_P(x_y) 1 sudden RebalanceStream
99.76631678685126
91.03442952492216
sine_P(x_y) 2 incremental improved_OOB
99.53769611451648
98.44605247119935
sine_P(x_y) 2 incremental WEOB1
99.32692419837262
98.74404651965804
sine_P(x_y) 2 incremental OOB
99.35725902085065
98.76376304163692
sine_P(x_y) 2 incremental ESOS_ELM
98.64702614919943
73.09943755865302
sine_P(x_y) 2 incremental improved_UOB
98.41037283670006
98.57960271809075
sine_P(x_y) 2 incremental WEOB2
98.11111395178298
98.7176547720214
sine_P(x_y) 2 incremental UOB
97.28676944452528
99.17800840739429
sine_P(x_y) 2 incremental OSHAT
98.0569683457621
96.31727503381966
sine_P(x_y) 2 i

In [5]:
old = final_dict.copy()

# NEW

In [6]:
final_dict = old.copy()

In [7]:
for new in new_drifts:
    path = main_dir + "/" + new 
    
    if new not in final_dict:
        final_dict[new] = {}
        
    for unbalance in range(1,5):
        if unbalance not in final_dict[new]:
            final_dict[new][unbalance] = {}


        path1 = path + "/" + str(unbalance)

        for alg in os.listdir(path1):
            if "DS_Store" in alg:
                continue
            path2 = path1 + "/" + alg
            
            alg_dtypes = {}
            alg_dtypes["incremental"] = {}
            alg_dtypes["sudden"] = {}
            alg_dtypes["periodic"] = {}
            alg_dtypes["incremental"]["class0"] = []
            alg_dtypes["sudden"]["class0"] = []
            alg_dtypes["periodic"]["class0"] = []
            alg_dtypes["incremental"]["class1"] = []
            alg_dtypes["sudden"]["class1"] = []
            alg_dtypes["periodic"]["class1"] = []
            
            
            print(f"{new} {unbalance} {alg}")
            #prendo i valori di tutti i 10 esperimenti dei 3 dtype
            
            for result in os.listdir(path2):
                if "err" not in result or "DS_Store" in result:
                    continue
                daily0 = []
                daily1 = []
                data = import_csv(path2+"/"+ result)
                start = False
                
                for data_row in data:
                    if data_row[1] == "learning evaluation instances":
                        start = True
                        continue
                    if not start:
                        continue

                    try:
                        daily0.append(float(data_row[11]))
                    except:
                        if data_row[11] == "?":
                            daily0.append(0.0)
                        else:
                            print("ZERO")
                            print(data_row)
                    try:
                        daily1.append(float(data_row[12]))
                    except:
                        if data_row[12] == "?":
                            daily1.append(0.0)
                        else:
                            print("ONE")
                            print(data_row)
                        
                    
                
                try:
                    if len(daily0)>0 and len(daily1)>0:
                        if alg == "ESOS_ELM" and new == "appearing-minority":
                            print(daily0)
                            print(daily1)
                        alg_dtypes[result[:result.index("_")]]["class0"].append(np.mean(daily0))
                        alg_dtypes[result[:result.index("_")]]["class1"].append(np.mean(daily1))
                except:
                    print(f"error with {result}")
            for k,i in alg_dtypes.items():      
                if len(i["class0"]) < 10:
                    print(f"N EXP {len(i['class0'])} in 0")
                if len(i["class1"]) < 10:
                    print(f"N EXP {len(i['class1'])} in 1")
                if k not in final_dict[new][unbalance]:
                    final_dict[new][unbalance][k] = {}
                assert alg not in final_dict[new][unbalance][k]
                final_dict[new][unbalance][k][alg] = {}
                final_dict[new][unbalance][k][alg]["class0"] ={} 
                final_dict[new][unbalance][k][alg]["class1"] = {}
                final_dict[new][unbalance][k][alg]["class0"]["mean"] = np.mean(i["class0"])
                final_dict[new][unbalance][k][alg]["class1"]["mean"] = np.mean(i["class1"])
                final_dict[new][unbalance][k][alg]["class0"]["std"] =np.std(i["class0"])
                final_dict[new][unbalance][k][alg]["class1"]["std"] = np.std(i["class1"])
                print(k)
                print(np.mean(i["class1"]))
                print(np.mean(i["class0"]))

jitter 1 improved_OOB
incremental
84.19652822434497
87.69715766404269
sudden
92.90569351207924
92.67779661705663
periodic
84.19652822434497
87.69715766404269
jitter 1 WEOB1
incremental
90.686076017918
83.47875328621868
sudden
94.81785190697101
90.76220956465252
periodic
90.686076017918
83.47875328621868
jitter 1 CSMOTE
incremental
82.28000932164011
77.14046903044438
sudden
90.08815293429414
79.46468104947641
periodic
82.28000932164011
77.14046903044438
jitter 1 OOB
incremental
87.92418826112012
84.96637512977597
sudden
94.74064085868244
90.84260544291475
periodic
87.92418826112012
84.96637512977597
jitter 1 ESOS_ELM
incremental
71.75083386190059
76.02777557779979
sudden
72.87731949004618
90.16423306275671
periodic
70.65448726440661
75.77752556912996
jitter 1 improved_UOB
incremental
92.26968311185553
74.59694398086074
sudden
94.91879033386125
80.80295454202215
periodic
92.26968311185553
74.59694398086074
jitter 1 WEOB2
incremental
92.84964582509829
73.39599431229318
sudden
94.432280842

incremental
53.30214055447398
86.92578753537376
sudden
48.211231463564886
87.6137476835123
periodic
97.2852801607744
80.31538644051184
disappearing-minority 3 CSMOTE
incremental
51.69099700312985
87.80186778504482
sudden
47.739836611935914
88.07301689518354
periodic
94.05434587216095
79.87240931203141
disappearing-minority 3 OOB
incremental
54.06223086746596
87.387203294468
sudden
49.030863758281825
87.55943696619579
periodic
97.55890878063403
79.43462195925322
disappearing-minority 3 ESOS_ELM
incremental
29.35655974898961
90.18873038787264
sudden
23.37664233512443
92.01667743707497
periodic
70.0084945538881
79.11953831141679
disappearing-minority 3 improved_UOB
incremental
53.371447649635705
75.18357952730807
sudden
48.295074788467446
73.45711700485825
periodic
97.80768719212766
78.16831227732096
disappearing-minority 3 WEOB2
incremental
52.953753699948166
75.27764264268657
sudden
47.846918928879816
72.423912438814
periodic
97.36315968293164
77.24218225385702
disappearing-minority 3 U

incremental
52.02621334402672
90.95679023684946
sudden
48.23527790111376
90.16386665769213
periodic
8.285126144026975
98.34896890525582
appearing-minority 2 CSMOTE
incremental
45.80367838129287
90.96511594973137
sudden
45.337514356373234
89.11375761315979
periodic
7.186540372109827
99.15454633917116
appearing-minority 2 OOB
incremental
50.7176889166319
92.31857186686497
sudden
48.38785611460689
90.40497106836372
periodic
7.789713541666667
99.37293487698044
appearing-minority 2 ESOS_ELM
[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[100.0, 100.0, 100.0, 100

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

incremental
93.93825555408998
88.9645193557885
sudden
94.26054676967547
88.89393950264557
periodic
99.37836703986997
92.55585825568185
appearing-clusters 1 CSMOTE
incremental
83.41925499644637
92.09555679914398
sudden
83.09429435614018
92.61397438258577
periodic
94.34166131724001
93.87759888729735
appearing-clusters 1 OOB
incremental
93.25117233127105
90.20541452961018
sudden
93.82570447526324
89.16766189852544
periodic
98.8103940198308
93.24559075233358
appearing-clusters 1 ESOS_ELM
incremental
71.23535981211454
79.39439486379386
sudden
70.20734612701176
79.5557568880689
periodic
76.49615971731848
81.82368670377413
appearing-clusters 1 improved_UOB
incremental
94.47129264364278
86.88645608864927
sudden
94.49768658989612
86.69372110156765
periodic
99.62189609236795
90.70320628646007
appearing-clusters 1 WEOB2
incremental
91.26743040093699
87.04983268714399
sudden
89.20313607838
87.03600573961737
periodic
99.68404545715211
89.8507687787093
appearing-clusters 1 UOB
incremental
93.7623728

incremental
95.49366995877583
79.6666570363644
sudden
95.20357502541552
79.8861499501936
periodic
96.08075078331855
78.99787112237235
shapeshift 3 CSMOTE
incremental
93.57722267509442
78.45020519994073
sudden
93.26233002997033
78.70578442427163
periodic
94.8721079203165
76.7728929064513
shapeshift 3 OOB
incremental
97.18827863131264
79.03347686922442
sudden
97.23568840534372
78.93638480905256
periodic
97.84584634540633
77.73669840333127
shapeshift 3 ESOS_ELM
incremental
71.69591432894627
78.45630581052372
sudden
71.00208190310312
78.62449547573722
periodic
68.73153076434686
77.73608536404643
shapeshift 3 improved_UOB
incremental
95.4600803117513
77.616285635744
sudden
95.24789514669992
77.67646863042019
periodic
96.01490419125915
76.56428490845371
shapeshift 3 WEOB2
incremental
95.41322746991429
76.62815245616488
sudden
95.46176219411196
77.14143627478875
periodic
95.14082936496501
76.75040609755547
shapeshift 3 UOB
incremental
97.08801101046228
74.60170427237695
sudden
96.947474576722

incremental
92.52529480236805
88.91401394761229
sudden
91.94072930945846
89.06127111709051
periodic
93.78676866953904
92.2546529849815
splitting-clusters 1 ESOS_ELM
incremental
72.35406354630332
80.14418151224334
sudden
68.6921232417563
79.83181106474575
periodic
75.81629950541449
82.58510775080228
splitting-clusters 1 improved_UOB
incremental
94.24618654632368
86.38048945797523
sudden
90.40374126548336
85.93919342800083
periodic
96.3386236722283
89.54703363596329
splitting-clusters 1 WEOB2
incremental
90.92713746920398
87.38246493305166
sudden
87.75242008494186
86.98490584525243
periodic
96.3828438984664
89.46418462657725
splitting-clusters 1 UOB
incremental
94.46990148069789
84.91686233199015
sudden
89.48159939369683
82.8866963741797
periodic
96.34173633347349
89.11371178293095
splitting-clusters 1 RebalanceStream
incremental
50.61785952605273
98.75090922272433
sudden
55.168527125815686
98.36144119754314
periodic
64.70004203648764
98.16162875350383
splitting-clusters 2 improved_OOB
i

incremental
90.95548052894313
73.03297083307936
sudden
79.957304972211
72.99251027280204
periodic
93.40079950957296
75.1809002149198
clusters-movement 3 UOB
incremental
97.22540650273672
69.60451547337757
sudden
95.28600746580379
63.23168021794457
periodic
95.07670847930578
71.81369882042291
clusters-movement 3 RebalanceStream
incremental
74.7197945294064
87.2404121685011
sudden
58.431368683250945
89.31957397793994
periodic
75.48245035656339
84.67107585920141
clusters-movement 4 improved_OOB
incremental
94.1799831670929
80.49395733206708
sudden
94.75560723846591
80.0158373280261
periodic
94.2256339077414
79.0001088980517
clusters-movement 4 WEOB1
incremental
94.74804487410981
79.51199711550768
sudden
94.87155591570391
79.29987397861962
periodic
94.39418269245172
78.16764006330303
clusters-movement 4 CSMOTE
incremental
89.02018462015134
70.66914596698334
sudden
84.9164052034711
71.26803518307634
periodic
92.98740434867412
77.93621104725992
clusters-movement 4 OOB
incremental
94.18838618

In [8]:
final_dict

{'sea_P(y)': {1: {'sudden': {'improved_OOB': {'class0': {'mean': 98.48561710677316,
      'std': 1.4210854715202004e-14},
     'class1': {'mean': 99.49352354854804, 'std': 0.0}},
    'WEOB1': {'class0': {'mean': 98.33357405890271, 'std': 0.0},
     'class1': {'mean': 99.6169731540414, 'std': 0.0}},
    'OOB': {'class0': {'mean': 98.3973579253213,
      'std': 1.4210854715202004e-14},
     'class1': {'mean': 99.44375523661355, 'std': 0.0}},
    'ESOS_ELM': {'class0': {'mean': 97.36586030796744,
      'std': 0.19451966631874398},
     'class1': {'mean': 74.88531568967352, 'std': 0.34930759356982527}},
    'improved_UOB': {'class0': {'mean': 98.06879810369382,
      'std': 1.4210854715202004e-14},
     'class1': {'mean': 99.5261267858681, 'std': 0.0}},
    'WEOB2': {'class0': {'mean': 97.74454272188926, 'std': 0.0},
     'class1': {'mean': 99.44302542805681, 'std': 0.0}},
    'UOB': {'class0': {'mean': 98.3973579253213,
      'std': 1.4210854715202004e-14},
     'class1': {'mean': 99.4437

In [4]:
# def stats_of(_dict ):
#     values_0 = []
#     values_1 = []
#     for alg, values in _dict.items():
#         if "ELM" in alg:
#             continue
#         values_0.append(values["class0"])
#         values_1.append(values["class1"])
#     return np.mean(values_0),np.mean(values_1), np.max(values_0), np.max(values_1), np.min(values_0), np.min(values_1)


In [9]:

with open('Recall_withSTD.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["drift","unbalance","type","alg","Recall_0", "Recall_1","Recall_0_std", "Recall_1_std"])
    for drift, dicts in final_dict.items():
        for unbalance, dtypes in dicts.items():
            for dtype, algs in dtypes.items():
#                 mean0, mean1,max0,max1,min0,min1 = stats_of(algs)
                for alg, values in algs.items():

                    print([drift,unbalance,dtype,alg])
                    print(values)
                    writer.writerow([drift,unbalance,dtype,alg,values["class0"]["mean"],values["class1"]["mean"],values["class0"]["std"],values["class1"]["std"]])

['sea_P(y)', 1, 'sudden', 'improved_OOB']
{'class0': {'mean': 98.48561710677316, 'std': 1.4210854715202004e-14}, 'class1': {'mean': 99.49352354854804, 'std': 0.0}}
['sea_P(y)', 1, 'sudden', 'WEOB1']
{'class0': {'mean': 98.33357405890271, 'std': 0.0}, 'class1': {'mean': 99.6169731540414, 'std': 0.0}}
['sea_P(y)', 1, 'sudden', 'OOB']
{'class0': {'mean': 98.3973579253213, 'std': 1.4210854715202004e-14}, 'class1': {'mean': 99.44375523661355, 'std': 0.0}}
['sea_P(y)', 1, 'sudden', 'ESOS_ELM']
{'class0': {'mean': 97.36586030796744, 'std': 0.19451966631874398}, 'class1': {'mean': 74.88531568967352, 'std': 0.34930759356982527}}
['sea_P(y)', 1, 'sudden', 'improved_UOB']
{'class0': {'mean': 98.06879810369382, 'std': 1.4210854715202004e-14}, 'class1': {'mean': 99.5261267858681, 'std': 0.0}}
['sea_P(y)', 1, 'sudden', 'WEOB2']
{'class0': {'mean': 97.74454272188926, 'std': 0.0}, 'class1': {'mean': 99.44302542805681, 'std': 0.0}}
['sea_P(y)', 1, 'sudden', 'UOB']
{'class0': {'mean': 98.3973579253213, 

{'class0': {'mean': 99.4607016587662, 'std': 0.0}, 'class1': {'mean': 22.086411165059392, 'std': 0.0}}
['disappearing-minority', 2, 'incremental', 'improved_OOB']
{'class0': {'mean': 89.22855900337251, 'std': 0.0}, 'class1': {'mean': 51.47150130301052, 'std': 0.0}}
['disappearing-minority', 2, 'incremental', 'WEOB1']
{'class0': {'mean': 85.05739821125812, 'std': 0.0}, 'class1': {'mean': 52.2668371642571, 'std': 0.0}}
['disappearing-minority', 2, 'incremental', 'CSMOTE']
{'class0': {'mean': 88.93573899389239, 'std': 0.0}, 'class1': {'mean': 48.75648501416088, 'std': 0.0}}
['disappearing-minority', 2, 'incremental', 'OOB']
{'class0': {'mean': 88.34594325353473, 'std': 0.0}, 'class1': {'mean': 53.26040820394138, 'std': 0.0}}
['disappearing-minority', 2, 'incremental', 'ESOS_ELM']
{'class0': {'mean': 90.68561625132259, 'std': 0.1470738116969886}, 'class1': {'mean': 29.25617737574342, 'std': 0.20687027588228193}}
['disappearing-minority', 2, 'incremental', 'improved_UOB']
{'class0': {'mean'

{'class0': {'mean': 80.49629460023883, 'std': 0.37549964038171246}, 'class1': {'mean': 68.19646642728591, 'std': 1.2674193221299965}}
['minority-share', 4, 'sudden', 'improved_UOB']
{'class0': {'mean': 78.64545979861637, 'std': 0.0}, 'class1': {'mean': 97.62708486373984, 'std': 1.4210854715202004e-14}}
['minority-share', 4, 'sudden', 'WEOB2']
{'class0': {'mean': 77.18356943804271, 'std': 0.0}, 'class1': {'mean': 97.342245299717, 'std': 0.0}}
['minority-share', 4, 'sudden', 'UOB']
{'class0': {'mean': 81.98139438246457, 'std': 0.0}, 'class1': {'mean': 95.54314959934206, 'std': 0.0}}
['minority-share', 4, 'sudden', 'RebalanceStream']
{'class0': {'mean': 81.71623127902193, 'std': 0.0}, 'class1': {'mean': 93.36505187336523, 'std': 0.0}}
['minority-share', 4, 'periodic', 'improved_OOB']
{'class0': {'mean': 87.48166702593552, 'std': 0.0}, 'class1': {'mean': 52.17158214799156, 'std': 0.0}}
['minority-share', 4, 'periodic', 'WEOB1']
{'class0': {'mean': 86.93951302856995, 'std': 1.42108547152020

# REALS

In [10]:
real_dict = {}
real_dict["KDDCup"] = {}
real_dict["PAKDD"] = {}
real_dict["Elec"] = {}

for i in real_dict.keys():
    path = f"/Users/08volt/Desktop/StremingML/TEST/Results_union/Real/{i}"
    for alg in os.listdir(path):
        class0 = []
        if "DS_Store" in alg or "ipynb" in alg:
            continue
        class1 = []
        if alg not in real_dict[i]:
            real_dict[i][alg] = {}
        
            
        for exp in os.listdir(path + "/" + alg): 
            if "err" not in exp or "DS_Store" in exp:
                continue
                
            data = import_csv(path + "/" + alg + "/"+ exp)
            daily0 = []
            daily1 = []
            start = False

            for data_row in data:
                if data_row[1] == "learning evaluation instances":
                    start = True
                    continue
                if not start:
                    continue

                try:
                    daily0.append(float(data_row[11]))
                except:
                    if data_row[11] == "?":
                        daily0.append(0.0)
                    else:
                        print("ZERO")
                        print(data_row)
                try:
                    daily1.append(float(data_row[12]))
                except:
                    if data_row[12] == "?":
                        daily1.append(0.0)
                    else:
                        print("ONE")
                        print(data_row)
            try:
                if len(daily0)>0 and len(daily1)>0:
                    class0.append(np.mean(daily0))
                    class1.append(np.mean(daily1))
            except:
                print(f"error with {result}")
        real_dict[i][alg]["class0"] = {}
        real_dict[i][alg]["class1"] = {}
        real_dict[i][alg]["class0"]["mean"] = np.mean(class0)
        real_dict[i][alg]["class1"]["mean"] = np.mean(class1)
        real_dict[i][alg]["class0"]["std"] = np.std(class0)
        real_dict[i][alg]["class1"]["std"] = np.std(class1)
        
                
                

In [11]:
real_dict

{'KDDCup': {'improved_OOB': {'class0': {'mean': 97.15655556275475, 'std': 0.0},
   'class1': {'mean': 99.74249148061718, 'std': 1.4210854715202004e-14}},
  'WEOB1': {'class0': {'mean': 98.88129505568767, 'std': 0.0},
   'class1': {'mean': 99.10139585088145, 'std': 0.0}},
  'OOB': {'class0': {'mean': 96.9798926629041, 'std': 1.4210854715202004e-14},
   'class1': {'mean': 99.86829041088046, 'std': 1.4210854715202004e-14}},
  'ESOS_ELM': {'class0': {'mean': 100.0, 'std': 0.0},
   'class1': {'mean': 0.0, 'std': 0.0}},
  'improved_UOB': {'class0': {'mean': 98.4018521449992,
    'std': 1.4210854715202004e-14},
   'class1': {'mean': 96.02451645343186, 'std': 0.0}},
  'WEOB2': {'class0': {'mean': 98.21314448463167, 'std': 0.0},
   'class1': {'mean': 95.27573057484446, 'std': 1.4210854715202004e-14}},
  'UOB': {'class0': {'mean': 95.94248111874364, 'std': 1.4210854715202004e-14},
   'class1': {'mean': 99.42185439987873, 'std': 0.0}},
  'OSHAT': {'class0': {'mean': 97.40330962335665,
    'std': 

In [13]:
with open('RecallXRecall_real_withSTD.csv', 'w') as csv_file:   
    writer = csv.writer(csv_file)
    writer.writerow(["ds","alg","Recall_0", "Recall_1","Recall_0_std", "Recall_1_std"])
    for ds, dicts in real_dict.items():
        for alg, values in dicts.items():

            if alg == "OSHAT":
                alg = "CSMOTE"
            writer.writerow([ds,alg,values["class0"]["mean"],values["class1"]["mean"],values["class0"]["std"],values["class1"]["std"]])
